In [ ]:
import mlflow
import mlflow.sklearn
import numpy as np
import pandas as pd
import psycopg2
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
# Configurar conexión a la base de datos
conn = psycopg2.connect("dbname=mlflow_db user=mlflow password=mlflow_pass host=localhost")

In [ ]:
# Obtener datos desde PostgreSQL
df = pd.read_sql("SELECT * FROM dataset_table", conn)

X = df.drop("target", axis=1)
y = df["target"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Configurar MLflow
mlflow.set_tracking_uri("http://10.43.101.193:5000")
mlflow.set_experiment("Experimentación con RandomForest")

# Ejecutar múltiples experimentos
for n_estimators in range(10, 210, 10):
    with mlflow.start_run():
        model = RandomForestClassifier(n_estimators=n_estimators, random_state=42)
        model.fit(X_train, y_train)
        
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)

        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_metric("accuracy", accuracy)
        mlflow.sklearn.log_model(model, "model")

        print(f"Modelo con {n_estimators} árboles, accuracy: {accuracy:.4f}")
